In [2]:
import tensorflow as tf

# TensorFlow Autograph Issue

It appears that TensorFlow has issues when trying to calculate the gradient of the following computation. Let's try to find out why.

In [3]:
#Define function using tensorflow defaults
def log1pexp(x):
    return tf.math.log(1 + tf.exp(x))

In [4]:
#Set up the computation
x = tf.constant(100.)
y = log1pexp(x)
dy = tf.gradients(y, x)[0]

In [5]:
#Print out output of tensorflow calculation.
with tf.Session() as sess:  print(dy.eval()) 

nan


As you can see, a gradient that should be easy to calculate ends up producing a NaN value. This is actually due to the way that TensorFlow handles the natural logarithm; values passed to the tf.math.log method that are too large or small cause inf values to be produced.

In [6]:
#Evalaute the result of the exponentiation.
a = tf.exp(100.)
b = tf.math.log(tf.exp(100.))
c = tf.math.log(tf.exp(-100.))
with tf.Session() as sess:  print(a.eval(), b.eval(), c.eval()) 

inf inf -inf


Implementing a custom gradient as follows will allow us to produce the correct result. 

In [7]:
#Implement custom gradient.
@tf.custom_gradient
def log1pexp_fix(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1+e))
    return tf.math.log(1 + e), grad

In [8]:
#Compute correct gradient.
new_grad = log1pexp_fix(x)
dy = tf.gradients(new_grad, x)[0]
with tf.Session() as sess:  print(dy.eval()) 

1.0


In [9]:
# Run code to get the computational graph.
with tf.Session() as sess:
    writer = tf.summary.FileWriter("output", sess.graph)
    print(sess.run(dy))
    writer.close()

1.0
